In [1]:
import sys
sys.path.append("../..")

import time
from time import sleep
from datetime import datetime

import MetaTrader5 as mt5
import pandas as pd
from IPython.display import display
from lightweight_charts import JupyterChart
from IPython.display import clear_output, display

from sesto.fractal import high_low_finder
from sesto.utils import get_price_at_pnl, calculate_fee, calculate_position_size, get_pnl_at_price, convert_lots_to_usd
from sesto.metatrader.business import get_positions, send_market_order, modify_sl_tp
from sesto.metatrader.data import fetch_data_pos
from sesto.constants import MT5Timeframe

MetaTrader5 initialized successfully
MetaTrader 5 initialized successfully.


In [2]:
# PAIRS = ['NZDJPY', 'AUDJPY', 'CADJPY', 'USDJPY']
PAIRS = ['BITCOIN', 'ETHEREUM', 'SOLANA']
MAIN_TIMEFRAME = MT5Timeframe.M15

TP_PNL_MULTIPLIER = 0.5
SL_PNL_MULTIPLIER = -0.25
LEVERAGE = 500
DEVIATION = 20

TRAILING_STOP_STEPS = [
    {'trigger_pnl_multiplier': 4.00, 'new_sl_pnl_multiplier': 3.50},
    {'trigger_pnl_multiplier': 3.50, 'new_sl_pnl_multiplier': 3.00},
    {'trigger_pnl_multiplier': 3.00, 'new_sl_pnl_multiplier': 2.75},
    {'trigger_pnl_multiplier': 2.75, 'new_sl_pnl_multiplier': 2.50},
    {'trigger_pnl_multiplier': 2.50, 'new_sl_pnl_multiplier': 2.25},
    {'trigger_pnl_multiplier': 2.25, 'new_sl_pnl_multiplier': 2.00},
    {'trigger_pnl_multiplier': 2.00, 'new_sl_pnl_multiplier': 1.75},
    {'trigger_pnl_multiplier': 1.75, 'new_sl_pnl_multiplier': 1.50},
    {'trigger_pnl_multiplier': 1.50, 'new_sl_pnl_multiplier': 1.25},
    {'trigger_pnl_multiplier': 1.25, 'new_sl_pnl_multiplier': 1.00},
    {'trigger_pnl_multiplier': 1.00, 'new_sl_pnl_multiplier': 0.75},
    {'trigger_pnl_multiplier': 0.75, 'new_sl_pnl_multiplier': 0.45},
    {'trigger_pnl_multiplier': 0.50, 'new_sl_pnl_multiplier': 0.22},
    {'trigger_pnl_multiplier': 0.25, 'new_sl_pnl_multiplier': 0.12},
    {'trigger_pnl_multiplier': 0.12, 'new_sl_pnl_multiplier': 0.05},
    {'trigger_pnl_multiplier': 0.05, 'new_sl_pnl_multiplier': 0.02},
]

In [3]:
def calculate_trade_capital(symbol, volume_lots, leverage, price_open):
    position_size_usd = convert_lots_to_usd(symbol, volume_lots, price_open)
    capital_used = position_size_usd / leverage
    return capital_used

In [4]:
def have_open_positions_in_symbol(symbol):
    positions = get_positions()
    return symbol in positions['symbol'].values

In [5]:
VOLUME = 0.1

while True:
    clear_output(wait=True)

    for pair in PAIRS:
        if have_open_positions_in_symbol(pair):
            continue

        df = fetch_data_pos(pair, MAIN_TIMEFRAME, 5)
        df['fractal'] = high_low_finder(df)
        df['symbol'] = pair
        display(df.tail())

        price = mt5.symbol_info_tick(pair).ask
        new_trade_capital = calculate_trade_capital(pair, VOLUME, LEVERAGE, price)
        trade_size = calculate_position_size(new_trade_capital, LEVERAGE)
        fee = calculate_fee(position_size_usd=trade_size) 

        last_row = df.iloc[-1]

        if df['fractal'].iloc[-1] == 'top':
            sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=trade_size, leverage=LEVERAGE, entry_price=price, type='short')
            order = send_market_order(symbol=pair, volume=0.1, order_type='sell', sl=sl, deviation=DEVIATION)
            display(order)
        elif df['fractal'].iloc[-1] == 'bottom':
            sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=trade_size, leverage=LEVERAGE, entry_price=price, type='long')
            order = send_market_order(symbol=pair, volume=0.1, order_type='buy', sl=sl, deviation=DEVIATION)
            display(order)

    positions = get_positions()
    positions['time'] = pd.to_datetime(positions['time'], unit='s')
    positions['time_update'] = pd.to_datetime(positions['time_update'], unit='s')

    for index, position in positions.iterrows():
        display(position.to_frame().T)
        
        # Calculate the actual capital used for this trade
        trade_capital = calculate_trade_capital(position.symbol, position.volume, LEVERAGE, position.price_open)
        trade_size = calculate_position_size(trade_capital, LEVERAGE)
        current_pnl_percentage = (position.profit / trade_capital) * 100
        current_sl_pnl = get_pnl_at_price(position.sl, position.price_open, trade_size, LEVERAGE, 'long' if position.type == 0 else 'short')
        current_sl_pnl_percentage = (current_sl_pnl / trade_capital) * 100
        print(f'{position.time_update} - {position.symbol} - POSITION INFO - CURRENT PRICE: ${position.price_current:.3f} - OPEN PRICE: ${position.price_open:.3f} - TRADE CAPITAL: ${trade_capital:.3f} - CURRENT SL: ${position.sl:.3f}')
        print(f'{position.time_update} - {position.symbol} - PNL INFO - PNL: ${position.profit:.3f} or {current_pnl_percentage:.3f}% - PNL AT CURRENT SL: ${current_sl_pnl:.3f} or {current_sl_pnl_percentage:.3f}%')

        for trailing_step in TRAILING_STOP_STEPS:
            trigger_pnl_multiplier, new_sl_pnl_multiplier = trailing_step['trigger_pnl_multiplier'], trailing_step['new_sl_pnl_multiplier']

            price_at_trigger_pnl_multiplier = get_price_at_pnl(pnl_multiplier=trigger_pnl_multiplier, order_fee=fee, position_size_usd=trade_size, leverage=LEVERAGE, entry_price=position.price_open, type='long' if position.type == 0 else 'short')
            new_sl_at_new_sl_pnl_multiplier = get_price_at_pnl(pnl_multiplier=new_sl_pnl_multiplier, order_fee=fee, position_size_usd=trade_size, leverage=LEVERAGE, entry_price=position.price_open, type='long' if position.type == 0 else 'short')
            pnl_at_trigger = get_pnl_at_price(price_at_trigger_pnl_multiplier, position.price_open, trade_size, LEVERAGE, 'long' if position.type == 0 else 'short')
            pnl_at_new_sl = get_pnl_at_price(new_sl_at_new_sl_pnl_multiplier, position.price_open, trade_size, LEVERAGE, 'long' if position.type == 0 else 'short')

            pnl_threshold = trigger_pnl_multiplier * trade_capital
            print(f'{position.time_update} - {position.symbol} - CHECKING TRAILING SL STEP - TRIGGER: {trigger_pnl_multiplier:.3f} (${price_at_trigger_pnl_multiplier:.3f})(${pnl_at_trigger:.3f} PNL) - NEW SL: {new_sl_pnl_multiplier:.3f} (${new_sl_at_new_sl_pnl_multiplier:.3f})(${pnl_at_new_sl:.3f} PNL)')

            if position.profit >= pnl_threshold:
                old_sl_price = position['sl']

                # Long
                if position.type == 0:
                    new_sl_price = get_price_at_pnl(pnl_multiplier=new_sl_pnl_multiplier, order_fee=fee, position_size_usd=trade_capital, leverage=LEVERAGE, entry_price=position.price_open, type='long')
                    if new_sl_price > old_sl_price:
                        print(f'{position.time_update} - {position.symbol} - NEW SL: ${new_sl_price:.3f}')
                        modify_sl_tp(position.ticket, new_sl_price, position.tp)                # Short
                else:
                    new_sl_price = get_price_at_pnl(pnl_multiplier=new_sl_pnl_multiplier, order_fee=fee, position_size_usd=trade_capital, leverage=LEVERAGE, entry_price=position.price_open, type='short')
                    if new_sl_price < old_sl_price:
                        print(f'{position.time_update} - {position.symbol} - NEW SL: ${new_sl_price:.3f}')
                        modify_sl_tp(position.ticket, new_sl_price, position.tp)
                
                break
                # if new_sl_price:
                #     capital_gain_at_current_unrealized_pnl = (position.profit / trade_capital) * 100
                #     old_sl_price_diff = 100.00 if old_sl_price == 0.0 else (position.price_open / old_sl_price - 1) * 100
                #     new_sl_price_diff = (position.price_open / new_sl_price - 1) * 100
                #     pnl_at_old_sl = get_pnl_at_price(current_price=old_sl_price, entry_price=position.price_open, position_size_usd=trade_size, leverage=LEVERAGE, type='long' if position.type == 0 else 'short')
                #     pnl_at_new_sl = get_pnl_at_price(current_price=position.sl, entry_price=position.price_open, position_size_usd=trade_size, leverage=LEVERAGE, type='long' if position.type == 0 else 'short')
                #     capital_loss_at_old_sl = (pnl_at_old_sl / trade_capital) * 100
                #     capital_loss_at_new_sl= (pnl_at_new_sl / trade_capital) * 100

                #     print(f"{position.time_update} - {position.symbol} - TRAILING STOP - UNREALIZED PNL: ${position.profit:.3f} ({capital_gain_at_current_unrealized_pnl:.3f}% CAP) - OLD SL: ${old_sl_price:.3f} ({old_sl_price_diff:.3f}%)({capital_loss_at_old_sl:.3f}% CAP) - NEW SL: ${position.sl:.3f} ({new_sl_price_diff:.3f}%)({capital_loss_at_new_sl:.3f}% CAP)")
                #     break  # Exit the loop after adjusting the stop loss
        

    sleep(5)

No open positions found.


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
0,2024-10-01 11:15:00,63958.78,64011.64,63864.66,63930.63,519,0,0,0,BITCOIN
1,2024-10-01 11:30:00,63931.12,63987.79,63864.26,63870.76,932,0,0,bottom,BITCOIN
2,2024-10-01 11:45:00,63871.61,63970.76,63828.28,63952.88,650,0,0,bottom,BITCOIN
3,2024-10-01 12:00:00,63952.42,64099.95,63928.32,64077.90,960,0,0,0,BITCOIN
4,2024-10-01 12:15:00,64074.95,64085.80,63990.11,64048.37,817,0,0,0,BITCOIN


No open positions found.


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
0,2024-10-01 11:15:00,2641.30,2643.09,2640.15,2642.21,1099,0,0,0,ETHEREUM
1,2024-10-01 11:30:00,2642.36,2645.20,2637.91,2638.13,1228,0,0,0,ETHEREUM
2,2024-10-01 11:45:00,2638.02,2641.47,2634.34,2640.21,1137,236,0,bottom,ETHEREUM
3,2024-10-01 12:00:00,2640.04,2644.20,2638.17,2643.96,1233,0,0,0,ETHEREUM
4,2024-10-01 12:15:00,2644.01,2644.29,2638.29,2642.85,1055,0,0,0,ETHEREUM


No open positions found.


,time,open,high,low,close,tick_volume,spread,real_volume,fractal,symbol
0,2024-10-01 11:15:00,156.40,156.57,155.93,156.26,336,9,0,0,SOLANA
1,2024-10-01 11:30:00,156.27,156.56,156.01,156.14,497,12,0,0,SOLANA
2,2024-10-01 11:45:00,156.14,156.39,155.86,156.29,371,12,0,bottom,SOLANA
3,2024-10-01 12:00:00,156.26,157.09,156.11,157.09,423,12,0,0,SOLANA
4,2024-10-01 12:15:00,157.09,157.16,156.64,156.81,338,12,0,0,SOLANA


No open positions found.
